# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import numpy as np
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @QuintinoEster
#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [4]:
#Produto escolhido:
produto = 'Doritos'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [5]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [7]:
dados = pd.read_excel('Doritos.xlsx')
dados_treinamento = pd.read_excel('Doritos.xlsx', 'Treinamento')
dados_teste = pd.read_excel('Doritos.xlsx', 'Teste')

In [8]:
keyword_r = [" doritos ", " comprar ", " comprei ", ' compramos ',' vou ', ' comer ', ' bom ', ' gostoso ', ' satisfação ',' pedir ', ' pede ', ' pedi ', ' pega ', ' peguei ',' pegamos ', ' melhor ', ' tradicional ', ' sabores ',' sabor ',' real ', ' reais ', ' comer ',' come ',' dividir ',' meu ', ' vontade ', ' é ', ' eh ',' vida ', ' combina ',' bem ', ' queria ', ' quero ', ' comi ', ' feliz ',' felicidade ',' com ', ' gordo ', ' gorda ', ' promoção ',' desejo ',' comendo ',' engordo ',' engordar ',' juntos ',' juntas ',' juntamos ',' juntei ',' besteira ',' jantei ',' nacho ',' cips ',' ']
keyword_i = [" eu "," tô ",' e ',' a ',' o ',' no ',' na ',' triste ',' mágoas ',' triste ',' cheetos ',' verme ',' eai ',' rt ',' com ',' um ',' uma ',' ai ',' aí ',' foi ',' fomos ',' pq ',' porque ',' porquê ',' por ',' que ',' por ',' q ',' bolo ',' nem ',' vou ',' velho ',' amigo ',' moisés ',' pra ',' eu ',' contos ',' conto ',' dps ',' depois ',' falar ',' flr ',' pequeno ',' msm ',' mesmo ',' deixa ',' ver ',' sua ',' alma ',' arco ',' íris ',' acabar ',' problemas ',' desinstalar ',' doideiras ',' pesadaum ',' frequentar ',' além ',' serio ',' sério ',' vezes ',' calar ',' observar ',' fernando ',' bala ',' colégio ',' porra ',' imbicar ',' promo ',' ativado ',' eles ',' pedindo ',' tempo ',' todo ',' assinatura ',' bobó ', ' camarão ',' hot ',' movimentando ',' indústria ',' industria ',' god ',' of ',' war ',' terça ',' vocês ',' ! ',' , ',' . ',' : ','  ',' # ',' batatas ',' fritas ',' batata ',' frita ',' pasta ',' de ',' dente ',' buchinha ',' shampoo ',' escova ',' cabelo ',' isqueiro ',' broas ',' razão ',' aínda ',' bagulho ',' ave ',' mano ',' 140 ',' ouvindo ',' percebi ',' rica ',' roleplay ',' saco ',' enchendo ',' bruna ',' im ',' sorry ',' boo ',' xvideos ',' correr ',' coca ',' trincando ',' vendo ',' lúcifer ',' boy ',' etc ',' né ',' saberdes ',' programa ',' respondi ',' pizza ',' fas ',' ovinte ',' matiruclsa ',' amiga ',' encontrar ',' ful ',' fenomeno ',' animal ',' apenas ',' deus ',' cortar ',' unha ',' maos ',' maravilhosa ',' noming ',' some ',' meninas ',' estômago ',' mãe ',' gabriel ',' cream ',' cheese ',' fogo ',' mateus ',' porém ',' não ',' nó ',' cabeça ',' noscope ',' sacao ',' geladamas ',' filme ',' roda ',' narga ',' check ',' in ',' prova ',' pau ',' fb ',' dadinho ',' is ',' this ',' msmo ',' antivirus ',' desativa ',' tendo ',' opostunidade ',' oportunidade ',' rodrigo ',' caio ',' participando ',' jogo ',' coop ',' hora ',' agora ',' bring ',' me ',' locos ',' taco ',' tri ',' arregado ',' irmã ', 'há ',' quanto ',' tempo ',' otaku ',' madrugão ',' buxo',' apache ',' dias ',' fuck ',' celo ',' flagrado ',' padaria ',' taubaté ',' island ',' socorro ',' compensação ',' tera ',' berola ',' seculo ',' tbm ',' tmb ',' vídeo ',' marcelo ',' rezende ',' cura ',' montanha-russa ',' aliás ',' me ',' mandou ',' vegas ',' presunto ',' pqp ',' sorte ',' * ',' eats ',' grilhoes ',' alarma ',' tengo ',' liga ',' ir ',' escola ',' vida ',' pitty ',' whapp ',' treinar ',' perna ',' subzero ',' cigarro ',' avontss ',' ru ',' inclinação ',' 90° ',' 90º ',' montar ',' ombros ',' deformados ',' gravar ',' mima ',' mt ',' tadeu ',' cola ',' heineken ',' prezunic ',' sky ',' posto ',' ipiranga ',' adicionei ',' playlist ',' eusou ',' titio ',' severim ',' 2010 ',' professor ', ' universitário ',' kansas ',' cereais ',' oreos ',' 2 ',' meses ',' 12 ',' kg ',' ondee ',' puta ',' saudade ',' época ',' dinheiro ',' mau ',' requeijão ', ' novela ',' começou ',' engradado ']

In [9]:
#Remover palavras irrelevantes:
for b in range(0,300):
    
    # Selecionando seção útil -- passo 1:
    a = dados_treinamento.Treinamento[b]
    
    # Deletando palavras e caracteres inúteis -- passo 2:
    for caractere in keyword_i:
        a = a.replace(caractere, " ")   
        
    # Removendo espaços excessivos -- passo 3:
    a = a.split()
    a = " ".join(a)
    dados_treinamento.Treinamento[b] = a
    
#Mesmo raciocínio para palavras irrelevantes:    
for b in range(0,200):
    
    # Passo 1:
    a = dados_teste.Teste[b]
    
    # Passo 2:
    for caractere in keyword_i:
        a = a.replace(caractere, " ")
        
    # Passo 3:
    a = a.split()
    a = " ".join(a)
    dados_teste.Teste[b] = a

In [10]:
doritos_r = dados_treinamento.loc[(dados.Classificação == 'r')]
doritos_i = dados_treinamento.loc[(dados.Classificação == 'i')]

In [11]:
dados_teste['Testando'] = 'i'

In [12]:
# É necessário, agora, uma lista de palavras, para ser usada no cálculo da Probabilidade de uma palavra ser relevante 
#ou irrelevante, sabendo que o espaço amostral se trata de uma palavra:
lista_de_palavras = []

for a in dados_treinamento.Treinamento:
    a = a.split()
    
    for b in a:
        lista_de_palavras.append(b)
        
lista_de_palavras = set(lista_de_palavras)
lista_de_palavras_num = len(lista_de_palavras)

# Espaço amostral das Palavras Relevantes:
lista_de_palavras_r = []

for a in doritos_r.Treinamento:
    a = a.split()
    
    for b in a:
        lista_de_palavras_r.append(b)
lista_de_palavras_r_num = len(lista_de_palavras_r)    

# Espaço amostral das Palavras Irrelevantes:
lista_de_palavras_i = []

for a in doritos_i.Treinamento:
    a = a.split()
    
    for b in a:
        lista_de_palavras_i.append(b)
lista_de_palavras_i_num = len(lista_de_palavras_i)    

print (lista_de_palavras_num, lista_de_palavras_r_num, lista_de_palavras_i_num)

1177 1116 1298


In [13]:
#Porporção entre palavras irrelevantes e relevantes, segundo o classificador:
pocket = dados.iloc[:,-1]
pocket.value_counts(normalize = True)

i    0.576667
r    0.423333
Name: Classificação, dtype: float64

In [14]:
# Calculando probabilidades:
for a in range(0,200):
    b = dados_teste.Teste[a]
    b = b.split()
    Pp_r = 1
    Pp_i = 1
    
    for c in b:
        c1 = lista_de_palavras_r.count(c)
        c1_1 = c1 + 1
        c2 = lista_de_palavras_r_num + lista_de_palavras_num
        c3 = c1_1/c2
        Pp_r = Pp_r*c3
    
    for c in b:
        c1 = lista_de_palavras_i.count(c)
        c1_1 = c1 + 1
        c2 = lista_de_palavras_i_num + lista_de_palavras_num
        c3 = c1_1/c2
        Pp_i = Pp_i*c3
        
    Pp_r = Pp_r * Pp_r
    Pp_i = Pp_i * Pp_i
        
    if Pp_r >Pp_i:
        dados_teste.Testando[a] = 'i'
        
    else:
        dados_teste.Testando[a] = 'r'

In [15]:
dados_teste

Teste Classificação Testando
0    tá foda essa 😂 desde cedo praia bebendo agr be...             i        i
1               partido + birra + doritos + amor. 😍💑🇦🇷             i        r
2    @dede_doritos @lbrownvieira cancela não, tem é...             i        i
3    rt @annainglat: 3 doritos depois percebemos aq...             i        r
4            @dede_doritos so mais 1 aula chega berola             i        i
5    rt @cellbit: deixo aqui gif do meu artista pre...             i        i
6    @murteira17 @illovesafadeza @realidadesfavel @...             i        r
7           @dede_doritos cheg lah es sempre bem vindo             i        r
8                    @fbssxx levar hj teu doritos kkkk             i        i
9                            @doritos_theo te respondi             i        r
10                                  meu almoço doritos             i        r
11            @doritos_45 vai assistir #sdlive cretino             i        i
12   rt @xxtfmr: pensei eram doritos https://t.co/e...             i        r
13   agr fudi ter tirar carteira d motorista até fi...             i        i
14   @bacaliau ni republica matriculei, to doido es...             i        i
15        isso é recorrente n compro doritos faz 1 ano             i        i
16   rt @cejoy_negah: @fbssxx levar hj teu doritos ...             i        i
17   tostacos &gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt; dori...             i        i
18   rt @cellbit: só pelo meme se você curte os meu...             i        r
19   da acenada lá — acene tu, tu quer doritos http...             i        i
20   #doritosgaming @doritos_mx @gamerstylemx metal...             i        i
21   rt @maconhudo: quando termino comer doritos ht...             i        r
22           faço isso doritos https://t.co/qxdlpkcyui             i        r
23                                      vou comprar lá             i        i
24   @dede_doritos @lbrownvieira só percebemos grav...             i        r
25   twix, umas pringles and onion ou uns doritos c...             i        r
26   crlll leo chapado é engraçado 😂 tia fatima ele...             i        i
27      @shanemcmahon @wwe namiral man bate fuder nele             i        i
28                                     na fita doritos             i        r
29   rt @dougraz: quando putasso tenho fingir deboa...             i        r
..                                                 ...           ...      ...
170  você percebe é solitária quando fica casa cana...             r        r
171   se meu vô trazer doritos mim, ficar muito triste             r        r
172                rt @jessicanevesfe1: queria doritos             r        r
173         comprei 50 reais doritos doce viajar &lt;3             r        i
174               abri doritos pimenta horrível 😷😷😷😷😷😷             r        i
175             rt @porrajapah: saudades comer doritos             r        r
176           só queria doritos meu dia ficaria melhor             r        r
177  gente do céu, meu pai foto doritos, nutella, c...             r        i
178  rt @dudinhasilvaahh: to cm vontade come doritos ♥             r        i
179  @pedrinhog98 já comi, tu conhece n recuso comi...             r        r
180  rt @narutozin: qndo tiver filho vo adorar merc...             r        i
181  rt @grayssondick: se fosse querer vácuo compra...             r        i
182       luiz trouxe fini doritos p mim, ele é d+ ele             r        i
183             saudade comer doritos leite condensado             r        r
184                          mol tempão como doritos 😋             r        r
185                                 queria uns doritos             r        r
186  com direito a: luís como guia turístico chuva ...             r        r
187          doritos é tão bom deveria ser chamado pau             r        r
188                                       fome doritos             r        r
189  rt @narutozin: qndo tiver filho vo adorar merc...             r   

In [17]:
crosstab = (pd.crosstab(dados_teste.Testando, dados_teste.Classificação, margins=True, rownames=['Método Naive-Bayes'], colnames=['Classificação manual'], normalize='index')*100).round(decimals=0)
crosstab

Classificação manual     i     r
Método Naive-Bayes              
i                     62.0  38.0
r                     43.0  57.0
All                   50.0  50.0

###### De todos os tweets marcados como relevantes pelo classificador 47% são relevantes, sendo que a porcentagem de positivos verdadeiros é de 47%
###### De todos os tweets marcados como relevantes pelo classificador 53% não são relevantes, sendo que a porcentagem de positivos falsos é de 53%
###### De todos os tweets marcados como irrelevantes pelo classificador 91% são irrelevantes, sendo que a porcentagem de negativos verdadeiros é de 91%
###### De todos os tweets marcados como irrelevantes pelo classificador 9% não são relevantes, sendo que a porcentagem de negativos falsos é de 9% 